# Importowanie niezbędnych bibliotek

In [1]:
# Standardowe biblioteki Pythona
import hashlib
import os
import pickle
import time
from datetime import datetime

# Biblioteki naukowe i manipulacja danymi
import numpy as np
import pandas as pd
import plotly.express as px

# Biblioteki ML i uczenia głębokiego
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Biblioteki do pracy z dźwiękiem i sygnałami
import librosa

# Biblioteki do wizualizacji
import matplotlib.pyplot as plt
import seaborn as sns

# Biblioteki do przygotowania danych i oceny modelu
from joblib import Parallel, delayed
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
        # Tworzenie połączonego wykresu przy użyciu subplots
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Biblioteki specyficzne dla projektu
from create_data import download_and_save_dataset
from datasets import load_from_disk
from config import (
    BATCH_SIZE, DATASET_PATH, DROPOUT_RATE, EARLY_STOPPING_PATIENCE, 
    LEARNING_RATE, MAX_LENGTH, NUM_EPOCHS, SEED, WEIGHT_DECAY, MODEL_DIR
)
from helpers.augment_for_all_types import AugmentedAudioDataset
from helpers.early_stopping import EarlyStopping
from helpers.resnet_model_definition import AudioResNet
from helpers.utils import find_results_directory, read_results_from_files
from helpers.data_proccesing import read_emotion_results
from helpers.vizualization import generate_accuracy_comparison_plot, generate_emotion_visualizations

# Ustawienie seed dla powtarzalności wyników
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# Utworzenie katalogu dla wyników
results_dir = 'feature_comparison_results'
os.makedirs(results_dir, exist_ok=True)


In [2]:
# Sprawdź czy folder data istnieje i załaduj dataset
dataset_path = DATASET_PATH
if os.path.exists(dataset_path):
    try:
        print("Ładowanie datasetu z dysku...")
        dataset = load_from_disk(dataset_path)
    except Exception as e:
        print(f"Błąd podczas ładowania datasetu: {e}")
        print("Pobieranie datasetu ponownie...")
        dataset = download_and_save_dataset()
else:
    print("Folder 'data' nie istnieje. Pobieranie datasetu...")
    dataset = download_and_save_dataset()

Ładowanie datasetu z dysku...


In [3]:
def extract_features(audio_array, sr, feature_type, max_length=3.0, 
                     n_mels=128, n_mfcc=40, n_chroma=12, 
                     n_fft=2048, hop_length=512, normalize=True):
    """Ekstrakcja różnych cech z sygnału audio.
    
    Args:
        audio_array: Sygnał audio w formie tablicy numpy
        sr: Częstotliwość próbkowania
        feature_type: Typ cechy do ekstrakcji
        max_length: Maksymalna długość sygnału w sekundach
        n_mels: Liczba pasm melowych dla melspektrogramu
        n_mfcc: Liczba współczynników MFCC
        n_chroma: Liczba pasm chromatycznych
        n_fft: Długość okna dla krótkoterminowej transformaty Fouriera
        hop_length: Przesunięcie okna między kolejnymi ramkami
        normalize: Czy normalizować wynikowe cechy
        
    Returns:
        Wyekstrahowane cechy w formie tablicy numpy
    """
    # Ujednolicenie długości
    target_length = int(max_length * sr)
    if len(audio_array) > target_length:
        audio_array = audio_array[:target_length]
    else:
        padding = np.zeros(target_length - len(audio_array))
        audio_array = np.concatenate([audio_array, padding])
    
    feature = None
    
    if feature_type == "melspectrogram":
        # Ekstrakcja melspektrogramu
        S = librosa.feature.melspectrogram(
            y=audio_array, sr=sr, n_mels=n_mels,
            n_fft=n_fft, hop_length=hop_length
        )
        feature = librosa.power_to_db(S, ref=np.max)
    
    elif feature_type == "spectrogram":
        # Standardowy spektrogram
        D = np.abs(librosa.stft(audio_array, n_fft=n_fft, hop_length=hop_length))
        feature = librosa.amplitude_to_db(D, ref=np.max)
    
    elif feature_type == "mfcc":
        # MFCC (Mel-frequency cepstral coefficients)
        feature = librosa.feature.mfcc(
            y=audio_array, sr=sr, n_mfcc=n_mfcc,
            n_fft=n_fft, hop_length=hop_length
        )
    
    elif feature_type == "chroma":
        # Chromagram
        feature = librosa.feature.chroma_stft(
            y=audio_array, sr=sr, n_chroma=n_chroma,
            n_fft=n_fft, hop_length=hop_length
        )
    
    elif feature_type == "spectral_contrast":
        # Spektralny kontrast
        feature = librosa.feature.spectral_contrast(
            y=audio_array, sr=sr, n_fft=n_fft, hop_length=hop_length
        )
    
    elif feature_type == "zcr":
    # Zero Crossing Rate
        feature = librosa.feature.zero_crossing_rate(
            audio_array, hop_length=hop_length
        )
        # Zawsze rozszerzaj wymiar dla ZCR
        expanded = np.zeros((n_mels, feature.shape[1]))
        normalized_feature = (feature - np.min(feature)) / (np.max(feature) - np.min(feature) + 1e-8)
        for i in range(n_mels):
            scale_factor = 1.0 - (i / float(n_mels))
            expanded[i, :] = normalized_feature * scale_factor
        feature = expanded

    elif feature_type == "rms":
        # RMS Energy
        feature = librosa.feature.rms(
            y=audio_array, hop_length=hop_length
        )
        # Zawsze rozszerzaj wymiar dla RMS
        expanded = np.zeros((n_mels, feature.shape[1]))
        normalized_feature = (feature - np.min(feature)) / (np.max(feature) - np.min(feature) + 1e-8)
        for i in range(n_mels):
            scale_factor = np.exp(-3.0 * (i / float(n_mels)))
            expanded[i, :] = normalized_feature * scale_factor
        feature = expanded

    elif feature_type == "tempogram":
        # Tempogram
        feature = librosa.feature.tempogram(
            y=audio_array, sr=sr, hop_length=hop_length
        )
    
    elif feature_type == "tonnetz":
        # Tonnetz - harmoniczne relacje
        y_harm = librosa.effects.harmonic(audio_array, margin=4.0)
        chroma = librosa.feature.chroma_cqt(
            y=y_harm, sr=sr, hop_length=hop_length
        )
        feature = librosa.feature.tonnetz(chroma=chroma, sr=sr)
    
    elif feature_type == "delta_mfcc":
        # Delta MFCC - zmiany w MFCC
        mfccs = librosa.feature.mfcc(
            y=audio_array, sr=sr, n_mfcc=n_mfcc,
            n_fft=n_fft, hop_length=hop_length
        )
        feature = librosa.feature.delta(mfccs)
    
    elif feature_type == "delta_tempogram":
        # Delta Tempogram - zmiany w tempie
        tempogram = librosa.feature.tempogram(
            y=audio_array, sr=sr, hop_length=hop_length
        )
        feature = librosa.feature.delta(tempogram)
    
    else:
        raise ValueError(f"Nieznany typ cechy: {feature_type}")
    
    # Normalizacja cech (opcjonalna, ale zalecana)
    if normalize and feature is not None:
        if feature_type in ["mfcc", "delta_mfcc"]:
            # Normalizacja MFCC - już zaimplementowana
            feature = librosa.util.normalize(feature)
        elif feature_type in ["melspectrogram", "spectrogram"]:
            # Dla spektrogramów - już przekształcone do dB
            pass
        else:
            # Standardowa normalizacja min-max dla pozostałych cech
            feature_min = np.min(feature)
            feature_max = np.max(feature)
            if feature_max > feature_min:
                feature = (feature - feature_min) / (feature_max - feature_min)
    
    return feature

In [4]:
def process_dataset(dataset, feature_type, max_length=3.0,
                    n_mels=128, n_mfcc=40, n_chroma=12,
                    n_fft=2048, hop_length=512, 
                    normalize_features=True, normalize_dataset=True,
                    n_jobs=-1, cache_dir="processed_features",
                    force_recompute=False, cv_folds=5):
    """
    Równoległe przetwarzanie całego zbioru danych audio na wybrany typ cechy z obsługą cache i walidacją krzyżową.
    
    Args:
        dataset: Zbiór danych zawierający próbki audio
        feature_type: Typ cechy do ekstrakcji
        max_length: Maksymalna długość próbki audio w sekundach
        n_mels: Liczba pasm melowych dla melspektrogramu
        n_mfcc: Liczba współczynników MFCC
        n_chroma: Liczba pasm chromatycznych
        n_fft: Długość okna FFT
        hop_length: Długość przeskoku między kolejnymi ramkami
        normalize_features: Czy normalizować pojedyncze cechy
        normalize_dataset: Czy normalizować cały zbiór danych
        n_jobs: Liczba równoległych procesów (-1 oznacza wszystkie dostępne rdzenie)
        cache_dir: Katalog do zapisywania przetworzonych cech
        force_recompute: Czy wymusić ponowne obliczenie cech, nawet jeśli istnieją w cache
        cv_folds: Liczba foldów do walidacji krzyżowej
        
    Returns:
        dict: Słownik zawierający dane treningowe, walidacyjne i testowe oraz metadane
    """


    # Tworzenie katalogu cache, jeśli nie istnieje
    os.makedirs(cache_dir, exist_ok=True)
    
    # Generowanie unikalnego ID dla tego zestawu parametrów
    params_str = f"{feature_type}_{max_length}_{n_mels}_{n_mfcc}_{n_chroma}_{n_fft}_{hop_length}_{normalize_features}_{normalize_dataset}_{cv_folds}"
    cache_id = hashlib.md5(params_str.encode()).hexdigest()
    cache_file = os.path.join(cache_dir, f"{feature_type}_{cache_id}.pkl")
    
    # Sprawdzenie, czy istnieje już plik cache
    if os.path.exists(cache_file) and not force_recompute:
        print(f"Wczytywanie przetworzonych cech z pliku cache: {cache_file}")
        with open(cache_file, 'rb') as f:
            return pickle.load(f)
    
    print(f"Przetwarzanie próbek audio dla cechy: {feature_type}...")
    start_time = time.time()
    
    # Przygotowanie danych do przetwarzania
    audio_samples = []
    all_labels = []
    sample_ids = []
    
    for i, sample in enumerate(dataset['train']):
        if i % 500 == 0:
            print(f"Przygotowywanie {i}/{len(dataset['train'])} próbek")
        sample_ids.append(i)
        audio_samples.append((sample['audio']['array'], sample['audio']['sampling_rate']))
        all_labels.append(sample['emotion'])
    
    # Funkcja do przetwarzania pojedynczej próbki
    def process_single_sample(i, audio_data):
        audio_array, sr = audio_data
        try:
            feature = extract_features(
                audio_array, sr, feature_type, max_length,
                n_mels=n_mels, n_mfcc=n_mfcc, n_chroma=n_chroma,
                n_fft=n_fft, hop_length=hop_length,
                normalize=normalize_features
            )
            
            if feature.size == 0 or (feature.ndim > 1 and feature.shape[1] == 0):
                return i, None, "Empty feature"
                
            return i, feature, None
            
        except Exception as e:
            return i, None, str(e)
    
    # Równoległe przetwarzanie próbek
    print(f"Rozpoczęcie równoległego przetwarzania na {n_jobs if n_jobs > 0 else 'wszystkich dostępnych'} rdzeniach...")
    results = Parallel(n_jobs=n_jobs)(
        delayed(process_single_sample)(i, audio_data) 
        for i, audio_data in enumerate(audio_samples)
    )
    
    # Zbieranie wyników
    processed_features = []
    valid_indices = []
    error_count = 0
    
    for i, feature, error in results:
        if feature is not None:
            processed_features.append(feature)
            valid_indices.append(i)
        else:
            error_count += 1
            # Dodanie logowania błędu, aby wiedzieć dlaczego próbka została odrzucona
            if i % 100 == 0 or "Empty feature" not in error:  # Loguj co 100 błędów lub niestandardowe błędy
                print(f"Błąd przy próbce {i}: {error}")
            
    # Konwersja list na tablice numpy
    if len(processed_features) == 0:
        raise ValueError(f"Nie udało się przetworzyć żadnych próbek dla cechy {feature_type}")
    
    features = np.array(processed_features)
    valid_labels = [all_labels[i] for i in valid_indices]
    
    # Przekształcenie do odpowiedniego formatu 4D: [próbki, kanały, wysokość, szerokość]
    features = features.reshape(features.shape[0], 1, features.shape[1], features.shape[2])
    
    # Kodowanie etykiet
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(valid_labels)
    num_classes = len(np.unique(encoded_labels))
    
    # Normalizacja całego zbioru danych (opcjonalnie)
    if normalize_dataset:
        mean = np.mean(features)
        std = np.std(features)
        if std > 0:
            features = (features - mean) / std
    
    # Tworzenie foldów dla walidacji krzyżowej
    skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
    cv_splits = list(skf.split(features, encoded_labels))
    
    # Przygotowanie słownika wynikowego
    result = {
        'feature_type': feature_type,
        'features': features,
        'labels': encoded_labels,
        'label_encoder': label_encoder,
        'num_classes': num_classes,
        'cv_splits': cv_splits,
        'params': {
            'max_length': max_length,
            'n_mels': n_mels,
            'n_mfcc': n_mfcc,
            'n_chroma': n_chroma,
            'n_fft': n_fft,
            'hop_length': hop_length,
            'normalize_features': normalize_features,
            'normalize_dataset': normalize_dataset
        },
        'processing_time': time.time() - start_time
    }
    
    # Wyświetl statystyki
    print(f"Całkowita liczba próbek: {len(audio_samples)}")
    print(f"Liczba ważnych próbek: {len(valid_indices)}")
    print(f"Liczba pustych/błędnych cech: {error_count}")
    print(f"Liczba klas emocji: {num_classes}")
    print(f"Mapowanie klas: {dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))}")
    print(f"Czas przetwarzania: {result['processing_time']:.2f} sekund")
    
    # Zapisz wyniki do cache
    print(f"Zapisywanie przetworzonych cech do pliku cache: {cache_file}")
    with open(cache_file, 'wb') as f:
        pickle.dump(result, f)
    
    return result

In [5]:
def train_with_cross_validation(dataset, feature_type, model_class=AudioResNet, 
                               batch_size=32, learning_rate=0.001, weight_decay=1e-5,
                               epochs=50, patience=10, n_jobs=-1, cache_dir="processed_features"):
    """
    Przeprowadza trening modelu z walidacją krzyżową.
    
    Args:
        dataset: Zbiór danych do przetwarzania
        feature_type: Typ cechy do ekstrakcji
        model_class: Klasa modelu do użycia
        batch_size: Rozmiar batcha
        learning_rate: Współczynnik uczenia
        weight_decay: Współczynnik regularyzacji
        epochs: Maksymalna liczba epok
        patience: Liczba epok bez poprawy, po której nastąpi early stopping
        n_jobs: Liczba równoległych procesów
        cache_dir: Katalog cache dla przetworzonych cech
        
    Returns:
        dict: Wyniki walidacji krzyżowej
    """

    
    # Przetwarzanie danych z walidacją krzyżową
    data = process_dataset(dataset, feature_type, n_jobs=n_jobs, cache_dir=cache_dir, cv_folds=5)
    
    features = data['features']
    labels = data['labels']
    cv_splits = data['cv_splits']
    num_classes = data['num_classes']
    
    # Wyniki dla każdego foldu
    cv_results = []
    
    # Wykrywanie urządzenia
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Używane urządzenie: {device}")
    
    # Przeprowadzenie treningu dla każdego foldu
    for fold, (train_idx, val_idx) in enumerate(cv_splits):
        print(f"\n{'=' * 30} Fold {fold+1}/{len(cv_splits)} {'=' * 30}")
        
        # Przygotowanie danych dla tego foldu
        X_train, X_val = features[train_idx], features[val_idx]
        y_train, y_val = labels[train_idx], labels[val_idx]
        
        # Konwersja do tensorów PyTorch
        X_train_tensor = torch.FloatTensor(X_train)
        y_train_tensor = torch.LongTensor(y_train)
        X_val_tensor = torch.FloatTensor(X_val)
        y_val_tensor = torch.LongTensor(y_val)
        
        # Tworzenie DataLoader
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)
        
        # Tworzenie modelu
        input_shape = features[0].shape
        model = model_class(input_shape, num_classes).to(device)
        
        # Definiowanie funkcji straty i optymalizatora
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
        
        # Śledzenie najlepszego modelu
        best_val_loss = float('inf')
        epochs_no_improve = 0
        best_epoch = 0
        best_model_state = None
        
        # Statystyki treningu
        history = {
            'train_loss': [],
            'val_loss': [],
            'val_acc': []
        }
        
        start_time = time.time()
        
        # Główna pętla treningu
        for epoch in range(epochs):
            # Tryb treningowy
            model.train()
            total_train_loss = 0
            for inputs, targets in train_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                
                # Zerowanie gradientów
                optimizer.zero_grad()
                
                # Forward pass
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                
                # Backward pass i optymalizacja
                loss.backward()
                optimizer.step()
                
                total_train_loss += loss.item() * inputs.size(0)
            
            # Obliczanie średniej straty treningowej
            avg_train_loss = total_train_loss / len(train_dataset)
            
            # Tryb ewaluacji
            model.eval()
            total_val_loss = 0
            correct = 0
            total = 0
            
            with torch.no_grad():
                for inputs, targets in val_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    
                    total_val_loss += loss.item() * inputs.size(0)
                    
                    _, predictions = torch.max(outputs, 1)
                    correct += (predictions == targets).sum().item()
                    total += targets.size(0)
            
            # Obliczanie średniej straty walidacyjnej i dokładności
            avg_val_loss = total_val_loss / len(val_dataset)
            val_accuracy = 100.0 * correct / total
            
            # Aktualizacja scheduler'a
            scheduler.step(avg_val_loss)
            
            # Zapisywanie historii
            history['train_loss'].append(avg_train_loss)
            history['val_loss'].append(avg_val_loss)
            history['val_acc'].append(val_accuracy)
            
            print(f"Epoka {epoch+1}/{epochs}, Strata treningu: {avg_train_loss:.4f}, "
                  f"Strata walidacji: {avg_val_loss:.4f}, Dokładność walidacji: {val_accuracy:.2f}%")
            
            # Sprawdzanie Early Stopping
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                best_epoch = epoch
                best_model_state = model.state_dict().copy()
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    print(f"Early stopping triggered! Brak poprawy przez {patience} epok.")
                    break
        
        # Czas treningu
        training_time = time.time() - start_time
        
        # Ładowanie najlepszego modelu
        model.load_state_dict(best_model_state)
        
        # Ewaluacja najlepszego modelu
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                _, predictions = torch.max(outputs, 1)
                correct += (predictions == targets).sum().item()
                total += targets.size(0)
        
        final_accuracy = 100.0 * correct / total
        
        # Zapisywanie wyników tego foldu
        fold_result = {
            'fold': fold + 1,
            'best_epoch': best_epoch + 1,
            'best_val_loss': best_val_loss,
            'final_accuracy': final_accuracy,
            'training_time': training_time,
            'history': history,
            'model_state': best_model_state
        }
        
        cv_results.append(fold_result)
        
        print(f"\nWyniki dla foldu {fold+1}:")
        print(f"Najlepsza epoka: {best_epoch+1}")
        print(f"Najlepsza strata walidacji: {best_val_loss:.4f}")
        print(f"Końcowa dokładność: {final_accuracy:.2f}%")
        print(f"Czas treningu: {training_time:.2f} sekund")
    
    # Obliczanie średnich wyników
    avg_accuracy = np.mean([res['final_accuracy'] for res in cv_results])
    avg_val_loss = np.mean([res['best_val_loss'] for res in cv_results])
    avg_training_time = np.mean([res['training_time'] for res in cv_results])
    
    print(f"\n{'=' * 30} Wyniki walidacji krzyżowej {'=' * 30}")
    print(f"Średnia dokładność: {avg_accuracy:.2f}% ± {np.std([res['final_accuracy'] for res in cv_results]):.2f}%")
    print(f"Średnia strata walidacji: {avg_val_loss:.4f}")
    print(f"Średni czas treningu: {avg_training_time:.2f} sekund")
    
    # Tworzenie słownika wynikowego
    final_results = {
        'feature_type': feature_type,
        'cv_results': cv_results,
        'avg_accuracy': avg_accuracy,
        'avg_val_loss': avg_val_loss,
        'avg_training_time': avg_training_time,
        'params': data['params'],
        'label_encoder': data['label_encoder'],
        'num_classes': num_classes
    }
    
    return final_results

In [6]:
def audio_collate_fn(batch):
    """
    Funkcja collate_fn dla DataLoader, z poprawioną obsługą ZCR i RMS.
    """
    features = []
    labels = []
    
    for feature, label in batch:
        # Pomiń None i tensory bez wymiarów
        if feature is None or feature.numel() == 0:
            continue
            
        # Upewnij się, że tensor ma prawidłowy format
        if feature.dim() == 2:  # Jeden wymiar + kanał
            # Rozszerz do 4D
            feature = feature.unsqueeze(0).unsqueeze(0)  # [H,W] -> [1,1,H,W]
        elif feature.dim() == 3:  # Cechy 2D bez kanału lub 1D z batch
            if feature.shape[0] == 1:  # Format [1, H, W]
                feature = feature.unsqueeze(0)  # [1,H,W] -> [1,1,H,W]
            else:  # Format [B, H, W]
                feature = feature.unsqueeze(1)  # [B,H,W] -> [B,1,H,W]
                
        features.append(feature)
        labels.append(label)
    
    # if len(features) == 0:
    #     print("Ostrzeżenie: Wszystkie elementy zostały odrzucone! Zwracam pusty tensor.")
    #     # Zwróć dummy tensor zamiast pustego
    #     return torch.zeros((1, 1, 4, 4)), torch.zeros(1, dtype=torch.long)
    
    # Dopełnij tensory do wspólnego rozmiaru
    try:
        max_height = max([f.shape[2] for f in features])
        max_width = max([f.shape[3] for f in features])
        
        for i in range(len(features)):
            if features[i].shape[2] < max_height or features[i].shape[3] < max_width:
                # Dopełnij zerami do pełnego rozmiaru
                padded = torch.zeros(features[i].shape[0], features[i].shape[1], 
                                    max_height, max_width, 
                                    device=features[i].device, dtype=features[i].dtype)
                padded[:, :, :features[i].shape[2], :features[i].shape[3]] = features[i]
                features[i] = padded
        
        features_batch = torch.cat(features, dim=0)
        labels_batch = torch.tensor(labels)
        
        return features_batch, labels_batch
    except Exception as e:
        print(f"Błąd podczas tworzenia batch: {e}")
        print(f"Kształty tensorów: {[f.shape for f in features]}")
        # Zwróć dummy tensor, aby zobaczyć gdzie jest problem
        return torch.zeros((1, 1, 4, 4)), torch.zeros(1, dtype=torch.long)

In [7]:
# Przygotowywanie danych z odpowiednią augmentacją
def prepare_datasets(X_train, X_val, X_test, y_train, y_val, y_test, feature_type, batch_size):
    """
    Przygotowuje zestawy danych z odpowiednią strategią augmentacji.
    """
    
    # Tworzenie datasetów z przekazaniem informacji o typie cechy
    train_dataset = AugmentedAudioDataset(
        X_train, y_train, 
        feature_type=feature_type,
        augment=True
    )
    
    val_dataset = AugmentedAudioDataset(
        X_val, y_val, 
        feature_type=feature_type,
        augment=False
    )
    
    test_dataset = AugmentedAudioDataset(
        X_test, y_test, 
        feature_type=feature_type,
        augment=False
    )
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        collate_fn=audio_collate_fn
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=batch_size, 
        collate_fn=audio_collate_fn
    )
    
    test_loader = DataLoader(
        test_dataset, 
        batch_size=batch_size, 
        collate_fn=audio_collate_fn
    )
    
    return train_loader, val_loader, test_loader

# Główna funkcja trenująca model dla wybranej cechy
def train_model_for_feature(dataset, feature_type, max_length=3.0):
    """
    Trenuje model ResNet dla wybranej reprezentacji dźwięku.
    
    Args:
        dataset: Dataset zawierający próbki audio i etykiety
        feature_type: Typ cechy do ekstrakcji (np. 'melspectrogram', 'mfcc')
        max_length: Maksymalna długość próbki audio w sekundach
    
    Returns:
        tuple: (model, test_loader, label_encoder, history, feature_dir, timestamp, feature_type, training_time)
    """
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    feature_dir = os.path.join(results_dir, feature_type)
    os.makedirs(feature_dir, exist_ok=True)
    
    # Przetwarzanie danych
    processed_data = process_dataset(dataset, feature_type, max_length)
    
    # Wyodrębnienie potrzebnych wartości ze słownika
    features = processed_data['features']
    labels = processed_data['labels']
    label_encoder = processed_data['label_encoder']
    num_classes = processed_data['num_classes']    
    # Podział na zbiory
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=SEED, stratify=labels)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=SEED, stratify=y_train)
    
    # Inicjalizacja modelu, funkcji straty i optymalizatora
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Używane urządzenie: {device}")
    
    model = AudioResNet(num_classes=num_classes, dropout_rate=DROPOUT_RATE)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
    
    # Wykorzystanie funkcji prepare_datasets do przygotowania danych
    train_loader, val_loader, test_loader = prepare_datasets(
        X_train, X_val, X_test, 
        y_train, y_val, y_test, 
        feature_type=feature_type,
        batch_size=BATCH_SIZE
    )
    
    # Ścieżka do zapisywania modelu
    model_path = os.path.join(feature_dir, f'best_model_{feature_type}_{timestamp}.pt')
    early_stopping = EarlyStopping(patience=EARLY_STOPPING_PATIENCE, path=model_path)
    
    # Śledzenie historii treningu
    history = {
        'train_loss': [],
        'val_loss': [],
        'val_accuracy': []
    }
    
    # Trenowanie modelu
    print(f"Rozpoczynanie treningu dla cechy: {feature_type}...")
    start_time = time.time()
    
    for epoch in range(NUM_EPOCHS):
        # Faza treningu
        model.train()
        running_loss = 0.0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        train_loss = running_loss/len(train_loader)
        history['train_loss'].append(train_loss)
        
        # Faza walidacji
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                
                # Obliczanie straty walidacyjnej
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                # Obliczanie dokładności
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * val_correct / val_total
        
        history['val_loss'].append(val_loss)
        history['val_accuracy'].append(val_accuracy)
        
        print(f'Epoka {epoch+1}/{NUM_EPOCHS}, Strata treningu: {train_loss:.4f}, '
              f'Strata walidacji: {val_loss:.4f}, Dokładność walidacji: {val_accuracy:.2f}%')
        
        # Aktualizacja schedulera
        scheduler.step(val_loss)
        
        # Sprawdzenie warunku early stopping
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping aktywowane!")
            break
    
    training_time = time.time() - start_time
    print(f"Trening zakończony po {epoch+1} epokach. Czas: {training_time:.2f} sekund")
    
    # Wczytanie najlepszego modelu
    model.load_state_dict(torch.load(model_path))
    
    return model, test_loader, label_encoder, history, feature_dir, timestamp, feature_type, training_time


In [8]:
def evaluate_model(model, test_loader, label_encoder, history, feature_dir, timestamp, 
                  feature_type, training_time, device=None, save_results=True):
    """
    Przeprowadza ewaluację modelu i generuje wizualizacje wyników.
    
    Args:
        model: Wytrenowany model
        test_loader: DataLoader z danymi testowymi
        label_encoder: Enkoder etykiet
        history: Historia treningu
        feature_dir: Katalog do zapisywania wyników
        timestamp: Znacznik czasowy
        feature_type: Typ cechy
        training_time: Czas treningu
        device: Urządzenie do ewaluacji (CPU/GPU)
        save_results: Czy zapisywać wyniki do plików
        
    Returns:
        tuple: (test_accuracy, history)
    """
    
    
    if device is None:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    criterion = nn.CrossEntropyLoss()
    model.eval()
    all_preds = []
    all_labels = []
    test_correct = 0
    test_total = 0
    test_loss = 0.0
    
    try:
        with torch.no_grad():
            for inputs, labels in test_loader:
                # Sprawdzenie czy batch nie jest pusty
                if inputs.numel() == 0 or labels.numel() == 0:
                    continue
                
                # Weryfikacja wymiarów danych wejściowych
                if inputs.dim() != 4:
                    continue
                
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                
                # Obliczanie straty testowej
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                
                # Obliczanie dokładności
                _, predicted = torch.max(outputs.data, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()
                
                # Zapisanie predykcji i rzeczywistych etykiet
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
        
        if len(test_loader) > 0:
            test_loss = test_loss / len(test_loader)
        else:
            return 0.0, history
            
        test_accuracy = 100 * test_correct / test_total if test_total > 0 else 0.0
        
        if save_results and all_preds and all_labels:
            # Macierz konfuzji
            cm = confusion_matrix(all_labels, all_preds)
            class_names = label_encoder.classes_
            
            # Wizualizacja macierzy konfuzji
            plt.figure(figsize=(10, 8))
            cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', 
                       xticklabels=class_names, yticklabels=class_names)
            plt.title(f'Znormalizowana macierz konfuzji - {feature_type}')
            plt.ylabel('Rzeczywista etykieta')
            plt.xlabel('Przewidziana etykieta')
            plt.tight_layout()
            plt.savefig(os.path.join(feature_dir, f'confusion_matrix_{feature_type}_{timestamp}.png'))
            plt.close()
            
            # Raport klasyfikacji
            report = classification_report(all_labels, all_preds, 
                                          target_names=class_names, output_dict=True)
            report_df = pd.DataFrame(report).transpose()
            report_df.to_csv(os.path.join(feature_dir, f'classification_report_{feature_type}_{timestamp}.csv'))
            
            # Wizualizacja historii treningu
            plt.figure(figsize=(12, 4))
            plt.subplot(1, 2, 1)
            plt.plot(history['train_loss'], label='Trening')
            plt.plot(history['val_loss'], label='Walidacja')
            plt.title(f'Strata podczas treningu - {feature_type}')
            plt.xlabel('Epoka')
            plt.ylabel('Strata')
            plt.legend()
            
            plt.subplot(1, 2, 2)
            plt.plot(history['val_accuracy'], label='Walidacja')
            plt.title(f'Dokładność podczas treningu - {feature_type}')
            plt.xlabel('Epoka')
            plt.ylabel('Dokładność (%)')
            plt.legend()
            
            plt.tight_layout()
            plt.savefig(os.path.join(feature_dir, f'training_history_{feature_type}_{timestamp}.png'))
            plt.close()
            
            # Zapisanie hiperparametrów i wyników
            results = {
                'feature_type': feature_type,
                'hyperparameters': {
                    'batch_size': BATCH_SIZE,
                    'initial_lr': LEARNING_RATE,
                    'weight_decay': WEIGHT_DECAY,   
                    'dropout_rate': DROPOUT_RATE,
                    'early_stopping_patience': EARLY_STOPPING_PATIENCE,
                    'max_epochs': NUM_EPOCHS,
                    'actual_epochs': len(history['train_loss'])
                },
                'performance': {
                    'test_accuracy': test_accuracy,
                    'test_loss': test_loss,
                    'val_accuracy': history['val_accuracy'][-1] if history['val_accuracy'] else None,
                    'val_loss': history['val_loss'][-1] if history['val_loss'] else None,
                    'training_time': training_time
                }
            }
            
            # Zapisanie wyników do pliku
            with open(os.path.join(feature_dir, f'results_{feature_type}_{timestamp}.txt'), 'w') as f:
                for section, values in results.items():
                    if isinstance(values, dict):
                        f.write(f"{section.upper()}:\n")
                        for key, value in values.items():
                            f.write(f"  {key}: {value}\n")
                        f.write("\n")
                    else:
                        f.write(f"{section}: {values}\n\n")
                        
        return test_accuracy, history
        
    except Exception as e:
        return 0.0, history

In [9]:
# Lista wszystkich cech do przetestowania
feature_types = [
    "spectrogram",
    "melspectrogram",
    "mfcc",
    "chroma",
    "spectral_contrast",
    "zcr",
    "rms",
    "tempogram",
    "tonnetz", 
    "delta_mfcc", 
    "delta_tempogram"  
]

def run_feature_comparison_experiment(dataset, feature_types_to_run=None, 
                                     skip_trained=True, save_interim=True,
                                     n_mels=128, n_mfcc=40, n_chroma=12,
                                     n_fft=2048, hop_length=512, 
                                     normalize_features=True, normalize_dataset=True):
    """
    Uruchamia eksperymenty dla różnych reprezentacji dźwięku i porównuje wyniki.
    
    Args:
        dataset: Zbiór danych do przetwarzania
        feature_types_to_run: Lista typów cech do uruchomienia (domyślnie wszystkie)
        skip_trained: Czy pomijać cechy, dla których istnieją już wyniki
        save_interim: Czy zapisywać wyniki częściowe po każdym typie cechy
        n_mels: Liczba pasm melowych dla melspektrogramu
        n_mfcc: Liczba współczynników MFCC
        n_chroma: Liczba pasm chromatycznych
        n_fft: Długość okna FFT
        hop_length: Długość przeskoku między kolejnymi ramkami
        normalize_features: Czy normalizować pojedyncze cechy
        normalize_dataset: Czy normalizować cały zbiór danych
        
    Returns:
        DataFrame z podsumowaniem wyników
    """
    
    # Katalog wyników
    results_dir = 'feature_comparison_results'
    os.makedirs(results_dir, exist_ok=True)
    
    # Inicjalizacja słownika wyników
    results = {}
    
    # Użyj przekazanej listy cech lub domyślnie wszystkich
    if feature_types_to_run is None:
        feature_types_to_run = feature_types
    
    # Sprawdź, czy istnieją wcześniejsze wyniki
    summary_path = os.path.join(results_dir, 'feature_comparison_summary.csv')
    if os.path.exists(summary_path) and skip_trained:
        try:
            existing_results = pd.read_csv(summary_path)
            trained_features = existing_results['Feature Type'].tolist()
            
            # Wczytaj istniejące wyniki
            for ft in trained_features:
                if ft in feature_types_to_run:
                    accuracy = existing_results[existing_results['Feature Type'] == ft]['Test Accuracy (%)'].values[0]
                    results[ft] = {
                        'accuracy': accuracy,
                        'history': None  # Historia nie jest zapisywana w CSV
                    }
            
            # Usuń już przetrenowane cechy z listy do uruchomienia
            feature_types_to_run = [ft for ft in feature_types_to_run if ft not in trained_features]
            
        except Exception as e:
            print(f"Nie udało się wczytać istniejących wyników: {e}")
    
    # Uruchamiaj eksperymenty dla każdego typu cechy
    start_time_all = time.time()
    
    for i, feature_type in enumerate(feature_types_to_run):
        try:
            # Trenowanie modelu z przekazaniem wszystkich parametrów
            model, test_loader, label_encoder, history, feature_dir, timestamp, feature_type, training_time = train_model_for_feature(
                dataset, feature_type, max_length=MAX_LENGTH,
                n_mels=n_mels, n_mfcc=n_mfcc, n_chroma=n_chroma,
                n_fft=n_fft, hop_length=hop_length,
                normalize_features=normalize_features, 
                normalize_dataset=normalize_dataset,
            )
            
            # Ewaluacja modelu
            device = next(model.parameters()).device
            accuracy, history = evaluate_model(
                model, test_loader, label_encoder, history, 
                feature_dir, timestamp, feature_type, 
                training_time, device
            )
            
            # Zapisz wyniki
            results[feature_type] = {
                'accuracy': accuracy,
                'history': history
            }
            
            # Zapisz częściowe wyniki, jeśli włączono tę opcję
            if save_interim:
                interim_results = {k: results[k]['accuracy'] for k in results}
                interim_df = pd.DataFrame({
                    'Feature Type': list(interim_results.keys()),
                    'Test Accuracy (%)': list(interim_results.values())
                }).sort_values('Test Accuracy (%)', ascending=False)
                
                interim_df.to_csv(os.path.join(results_dir, 'interim_results.csv'), index=False)
            
        except Exception as e:
            # Zapisz informację o błędzie
            results[feature_type] = {
                'accuracy': 0.0,
                'history': None,
                'error': str(e)
            }
    
    total_time = time.time() - start_time_all
    
    # Dodaj wcześniej przetrenowane cechy
    all_results = {}
    all_results.update(results)
    
    results_df = pd.DataFrame({
        'Feature Type': list(all_results.keys()),
        'Test Accuracy (%)': [all_results[ft]['accuracy'] for ft in all_results.keys()]
    })
    
    results_df = results_df.sort_values('Test Accuracy (%)', ascending=False)
    
    # Zapisz podsumowanie do pliku CSV
    results_df.to_csv(os.path.join(results_dir, 'feature_comparison_summary.csv'), index=False)
    
    # Wizualizacja porównania dokładności
    plt.figure(figsize=(12, 6))
    plt.bar(results_df['Feature Type'], results_df['Test Accuracy (%)'])
    plt.title('Porównanie dokładności dla różnych reprezentacji audio')
    plt.xlabel('Typ cechy')
    plt.ylabel('Dokładność testu (%)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(results_dir, 'accuracy_comparison.png'))
    plt.close()
    
    # Dodaj wizualizację czasu treningu jeśli dostępne
    if any('training_time' in all_results.get(ft, {}) for ft in all_results):
        times_df = pd.DataFrame({
            'Feature Type': [ft for ft in all_results if 'training_time' in all_results[ft]],
            'Training Time (s)': [all_results[ft].get('training_time', 0) for ft in all_results 
                                 if 'training_time' in all_results[ft]]
        })
        
        plt.figure(figsize=(12, 6))
        plt.bar(times_df['Feature Type'], times_df['Training Time (s)'])
        plt.title('Porównanie czasu treningu dla różnych reprezentacji audio')
        plt.xlabel('Typ cechy')
        plt.ylabel('Czas treningu (s)')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(results_dir, 'training_time_comparison.png'))
        plt.close()
    
    return results_df

In [10]:
def read_accuracy_from_results(feature_type, results_base_dir='feature_comparison_results'):
    """
    Odczytuje dokładność z zapisanych wyników dla określonego typu cechy.
    
    Args:
        feature_type: Typ cechy (np. 'mfcc', 'spectrogram')
        results_base_dir: Katalog bazowy zawierający wyniki
        
    Returns:
        Dokładność jako liczba zmiennoprzecinkowa lub None, jeśli nie znaleziono
    """
    import os
    import json
    
    feature_dir = os.path.join(results_base_dir, feature_type)
    
    if not os.path.exists(feature_dir):
        return None
    
    # Szukanie pliku results.json w podfolderach
    for root, dirs, files in os.walk(feature_dir):
        for file in files:
            if file == 'results.json':
                try:
                    with open(os.path.join(root, file), 'r') as f:
                        results = json.load(f)
                        # Sprawdź różne możliwe klucze dla dokładności
                        for key in ['accuracy', 'test_accuracy', 'val_accuracy']:
                            if key in results:
                                return float(results[key])
                except Exception as e:
                    print(f"Błąd podczas odczytu pliku {os.path.join(root, file)}: {str(e)}")
    
    return None

In [11]:
def run_training_experiment(dataset, feature_type=None, skip_trained=False, results_base_dir='feature_comparison_results'):
    """
    Funkcja uruchamiająca trening dla wybranej cechy lub wszystkich cech.
    
    Args:
        dataset: Zbiór danych do treningu
        feature_type: Konkretna cecha do treningu (None oznacza wszystkie cechy)
        skip_trained: Czy pomijać cechy, dla których istnieją już wyniki
        results_base_dir: Katalog bazowy do zapisywania wyników
    
    Returns:
        DataFrame z podsumowaniem wyników
    """

    
    # Lista wszystkich dostępnych typów cech
    all_feature_types = [
        "spectrogram", "melspectrogram", "mfcc", "chroma", 
        "spectral_contrast", "zcr", "rms", "tempogram",
        "tonnetz", "delta_mfcc", "delta_tempogram"
    ]
    
    # Ustal, które cechy trenować
    feature_types_to_train = [feature_type] if feature_type else all_feature_types
    
    # Tworzenie katalogów dla wyników
    os.makedirs(results_base_dir, exist_ok=True)
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Słowniki do przechowywania wyników i czasów treningu
    results = {}
    training_times = {}
    
    # Trening dla każdej cechy
    for feat_type in feature_types_to_train:
        # Sprawdź, czy już istnieją wyniki dla tej cechy
        feature_dir = os.path.join(results_base_dir, feat_type)
        
        if skip_trained and os.path.exists(feature_dir) and len(os.listdir(feature_dir)) > 0:
            print(f"\nPomijanie cechy {feat_type.upper()} - znaleziono istniejące wyniki.")
            
            # Odczytaj dokładność z istniejących wyników
            accuracy = read_accuracy_from_results(feat_type, results_base_dir)
            if accuracy:
                results[feat_type] = accuracy
                print(f"Odczytana dokładność: {accuracy:.2f}%")
            continue
        
        # Początek treningu dla danej cechy
        print(f"\n{'=' * 50}")
        print(f"Trening modelu na reprezentacji: {feat_type.upper()}")
        print(f"{'=' * 50}\n")
        
        # Pomiar czasu treningu
        start_time = time.time()
        
        # Trening modelu
        try:
            model, test_loader, label_encoder, history, feature_dir, feat_timestamp, _, training_time = train_model_for_feature(dataset, feat_type)
            
            # Ewaluacja modelu
            device = next(model.parameters()).device
            accuracy, _ = evaluate_model(model, test_loader, label_encoder, history, 
                                        feature_dir, feat_timestamp, feat_type, 
                                        time.time() - start_time, device)
            
            results[feat_type] = accuracy
            training_times[feat_type] = time.time() - start_time
            
            print(f"\nTrening dla {feat_type} zakończony sukcesem. Dokładność: {accuracy:.2f}%")
            print(f"Czas treningu: {training_times[feat_type]:.2f} sekund")
            
        except Exception as e:
            print(f"\nBłąd podczas treningu dla cechy {feat_type}: {str(e)}")
            continue
    
    # Generowanie raportu podsumowującego (tylko jeśli trenowano więcej niż jedną cechę)
    if len(results) > 1:
        print("\n" + "="*50)
        print("Wszystkie treningi zakończone. Generowanie raportu zbiorczego...")
        print("="*50 + "\n")
        
        # Tworzenie DataFrame z wynikami
        df = pd.DataFrame({
            'Feature Type': list(results.keys()),
            'Test Accuracy (%)': [results[ft] for ft in results.keys()],
            'Training Time (s)': [training_times.get(ft, 0) for ft in results.keys()]
        })
        
        # Sortowanie według dokładności
        df = df.sort_values('Test Accuracy (%)', ascending=False)
        
        # Zapis do CSV
        csv_path = os.path.join(results_base_dir, f'accuracy_summary_{timestamp}.csv')
        df.to_csv(csv_path, index=False)
        print(f"Zapisano podsumowanie do: {csv_path}")
        
        # Tworzenie wykresu dokładności przy użyciu Plotly
        fig_accuracy = px.bar(
            df, 
            x='Feature Type', 
            y='Test Accuracy (%)', 
            title='Porównanie dokładności dla różnych reprezentacji audio',
            color_discrete_sequence=['purple']  # Kolor fioletowy
        )
        
        fig_accuracy.update_layout(
            xaxis_title='Typ cechy',
            yaxis_title='Dokładność testu (%)',
            xaxis_tickangle=-45,
            yaxis_range=[0, max(df['Test Accuracy (%)']) * 1.1],
            template='plotly_white'
        )
        
        # Dodanie wartości nad słupkami
        fig_accuracy.update_traces(
            texttemplate='%{y:.1f}%', 
            textposition='outside'
        )
        
        # Zapisanie wykresu dokładności
        accuracy_plot_path = os.path.join(results_base_dir, f'accuracy_comparison_{timestamp}.html')
        fig_accuracy.write_html(accuracy_plot_path)
        
        # Tworzenie wykresu czasu treningu przy użyciu Plotly
        fig_time = px.bar(
            df, 
            x='Feature Type', 
            y='Training Time (s)', 
            title='Porównanie czasu treningu dla różnych reprezentacji audio',
            color_discrete_sequence=['purple']  # Kolor fioletowy
        )
        
        fig_time.update_layout(
            xaxis_title='Typ cechy',
            yaxis_title='Czas treningu (s)',
            xaxis_tickangle=-45,
            yaxis_range=[0, max(df['Training Time (s)']) * 1.1],
            template='plotly_white'
        )
        
        # Dodanie wartości nad słupkami
        fig_time.update_traces(
            texttemplate='%{y:.0f}s', 
            textposition='outside'
        )
        
        # Zapisanie wykresu czasu treningu
        time_plot_path = os.path.join(results_base_dir, f'training_time_comparison_{timestamp}.html')
        fig_time.write_html(time_plot_path)
    
        
        fig_combined = make_subplots(
            rows=1, cols=2,
            subplot_titles=('Porównanie dokładności dla różnych reprezentacji audio', 
                           'Porównanie czasu treningu dla różnych reprezentacji audio')
        )
        
        # Dodanie słupków dokładności
        fig_combined.add_trace(
            go.Bar(
                x=df['Feature Type'], 
                y=df['Test Accuracy (%)'],
                text=df['Test Accuracy (%)'].apply(lambda x: f'{x:.1f}%'),
                textposition='outside',
                marker_color='purple',
                name='Dokładność'
            ),
            row=1, col=1
        )
        
        # Dodanie słupków czasu treningu
        fig_combined.add_trace(
            go.Bar(
                x=df['Feature Type'], 
                y=df['Training Time (s)'],
                text=df['Training Time (s)'].apply(lambda x: f'{int(x)}s'),
                textposition='outside',
                marker_color='purple',
                name='Czas treningu'
            ),
            row=1, col=2
        )
        
        # Aktualizacja układu
        fig_combined.update_layout(
            height=600,
            width=1200,
            showlegend=False,
            template='plotly_white'
        )
        
        # Aktualizacja osi X i Y dla obu wykresów
        fig_combined.update_xaxes(title_text='Typ cechy', tickangle=-45, row=1, col=1)
        fig_combined.update_xaxes(title_text='Typ cechy', tickangle=-45, row=1, col=2)
        fig_combined.update_yaxes(title_text='Dokładność testu (%)', range=[0, max(df['Test Accuracy (%)']) * 1.1], row=1, col=1)
        fig_combined.update_yaxes(title_text='Czas treningu (s)', range=[0, max(df['Training Time (s)']) * 1.1], row=1, col=2)
        
        # Zapisanie połączonego wykresu
        combined_plot_path = os.path.join(results_base_dir, f'feature_comparison_{timestamp}.html')
        fig_combined.write_html(combined_plot_path)
        
        print(f"Zapisano interaktywne wizualizacje do: {accuracy_plot_path}, {time_plot_path}, {combined_plot_path}")
        print("\nPodsumowanie wyników:")
        print(df)
        
        return df
    
    # Jeśli trenowano tylko jedną cechę, zwróć wynik
    elif len(results) == 1:
        feature = list(results.keys())[0]
        print(f"\nWynik dla cechy {feature}: {results[feature]:.2f}%")
        return results[feature]
    
    # Jeśli nie trenowano żadnej cechy
    else:
        print("\nNie przeprowadzono żadnego treningu.")
        return None

In [12]:
# Uruchom trening dla wszystkich typów cech
# results_df = run_training_experiment(dataset)

In [13]:
# # Uruchom trening tylko dla melspektrogramu
# accuracy = run_training_experiment(dataset, feature_type="chroma")

In [12]:
# # Uruchom trening pomijając cechy, dla których już istnieją wyniki
results_df = run_training_experiment(dataset, skip_trained=True)


Pomijanie cechy SPECTROGRAM - znaleziono istniejące wyniki.

Pomijanie cechy MELSPECTROGRAM - znaleziono istniejące wyniki.

Pomijanie cechy MFCC - znaleziono istniejące wyniki.

Pomijanie cechy CHROMA - znaleziono istniejące wyniki.

Pomijanie cechy SPECTRAL_CONTRAST - znaleziono istniejące wyniki.

Pomijanie cechy ZCR - znaleziono istniejące wyniki.

Pomijanie cechy RMS - znaleziono istniejące wyniki.

Pomijanie cechy TEMPOGRAM - znaleziono istniejące wyniki.

Pomijanie cechy TONNETZ - znaleziono istniejące wyniki.

Pomijanie cechy DELTA_MFCC - znaleziono istniejące wyniki.

Pomijanie cechy DELTA_TEMPOGRAM - znaleziono istniejące wyniki.

Nie przeprowadzono żadnego treningu.


In [ ]:

def generate_all_visualizations():
    """Generuje i zapisuje wszystkie wizualizacje wyników."""
    # Znajdź katalog wyników
    base_dir = find_results_directory()
    if base_dir is None:
        return
    
    # Pobierz wyniki
    results_df = read_results_from_files(base_dir)
    emotions_df = read_emotion_results(base_dir)
    
    # Określ katalog zapisu
    save_dir = base_dir
    
    # Przetwórz wyniki dokładności
    if results_df is not None and not results_df.empty:
        # Sortowanie według dokładności malejąco
        results_df = results_df.sort_values('Test Accuracy (%)', ascending=False)
        
        # Wyświetl DataFrame dla podglądu
        print(f"\nZnalezione wyniki dokładności:\n{results_df}")
        
        # Zapisz DataFrame do CSV
        csv_path = os.path.join(save_dir, 'feature_comparison_summary_auto.csv')
        results_df.to_csv(csv_path, index=False)
        print(f"Zapisano wyniki dokładności do: {csv_path}")
        
        # Generuj wykres porównania dokładności
        combined_path = generate_accuracy_comparison_plot(results_df, save_dir)
        print(f"Zapisano wykres porównania dokładności do: {combined_path}")
    else:
        print("Brak danych dokładności do wygenerowania wykresów.")
    
    # Przetwórz wyniki emocji
    if emotions_df is not None and not emotions_df.empty:
        # Zapisz DataFrame do CSV
        emotions_csv_path = os.path.join(save_dir, 'emotions_comparison_auto.csv')
        emotions_df.to_csv(emotions_csv_path, index=False)
        print(f"\nZapisano wyniki emocji do: {emotions_csv_path}")
        
        # Generuj wizualizacje emocji
        emotion_paths = generate_emotion_visualizations(emotions_df, results_df, save_dir)
        print("\nWygenerowane wizualizacje emocji:")
        for name, path in emotion_paths.items():
            if path:
                print(f"- {name}: {path}")
        
        print("\nGenerowanie wszystkich wizualizacji zakończone pomyślnie.")
    else:
        print("Brak danych emocji do wygenerowania wykresów.")

# Uruchom generowanie wszystkich wizualizacji
generate_all_visualizations()